In [57]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import pickle

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [58]:
def eda_SR(originalSentence, n):
  """
  Paper Methodology -> Randomly choose n words from the sentence that are not stop words.
                       Replace each of these words with one of its synonyms chosen at random.
  originalSentence -> The sentence on which EDA is to be applied
  n -> The number of words to be chosen for random synonym replacement
  """
  stops = set(stopwords.words('indonesian'))
  splitSentence = list(originalSentence.split(" "))
  splitSentenceCopy = splitSentence.copy()
  # Since We Make Changes to The Original Sentence List The Indexes Change and Hence an initial copy proves useful to get values
  ls_nonStopWordIndexes = []
  for i in range(len(splitSentence)):
    if splitSentence[i].lower() not in stops:
      ls_nonStopWordIndexes.append(i)
  if (n > len(ls_nonStopWordIndexes)):
    raise Exception("The number of replacements exceeds the number of non stop word words")
  for i in range(n):
    indexChosen = random.choice(ls_nonStopWordIndexes)
    ls_nonStopWordIndexes.remove(indexChosen)
    synonyms = []
    originalWord = splitSentenceCopy[indexChosen]
    for synset in wordnet.synsets(originalWord):
      for lemma in synset.lemmas():
        if lemma.name() != originalWord:
          synonyms.append(lemma.name())
    if (synonyms == []):
      continue
    splitSentence[indexChosen] = random.choice(synonyms).replace('_', ' ')
  return " ".join(splitSentence)

In [59]:
print(eda_SR("halo admin untuk besok stock vaksin moderna ada ga ya estimasi dimana terimakasih", 6))

annulus admin untuk besok ancestry vaksin moderna ada Ga ya estimasi dimana terimakasih


In [60]:
def eda_RI(originalSentence, n):
  """
  Paper Methodology -> Find a random synonym of a random word in the sentence that is not a stop word.
                       Insert that synonym into a random position in the sentence. Do this n times
  originalSentence -> The sentence on which EDA is to be applied
  n -> The number of times the process has to be repeated
  """
  stops = set(stopwords.words('indonesian'))
  splitSentence = list(originalSentence.split(" "))
  splitSentenceCopy = splitSentence.copy()
  # Since We Make Changes to The Original Sentence List The Indexes Change and Hence an initial copy proves useful to get values
  ls_nonStopWordIndexes = []
  for i in range(len(splitSentence)):
    if splitSentence[i].lower() not in stops:
      ls_nonStopWordIndexes.append(i)
  if (n > len(ls_nonStopWordIndexes)):
    raise Exception("The number of replacements exceeds the number of non stop word words")
  WordCount = len(splitSentence)
  for i in range(n):
    indexChosen = random.choice(ls_nonStopWordIndexes)
    ls_nonStopWordIndexes.remove(indexChosen)
    synonyms = []
    originalWord = splitSentenceCopy[indexChosen]
    for synset in wordnet.synsets(originalWord):
      for lemma in synset.lemmas():
        if lemma.name() != originalWord:
          synonyms.append(lemma.name())
    if (synonyms == []):
      continue
    splitSentence.insert(random.randint(0,WordCount-1), random.choice(synonyms).replace('_', ' '))
  return " ".join(splitSentence)

In [61]:
print(eda_RI("halo admin untuk besok stock vaksin moderna ada ga ya estimasi dimana terimakasih", 6))

halo aureole admin untuk besok stock Peach State vaksin moderna ada ga ya estimasi dimana terimakasih


In [62]:
def eda_RS(originalSentence, n):
  """
  Paper Methodology -> Find a random synonym of a random word in the sentence that is not a stop word.
                       Insert that synonym into a random position in the sentence. Do this n times
  originalSentence -> The sentence on which EDA is to be applied
  n -> The number of times the process has to be repeated
  """
  splitSentence = list(originalSentence.split(" "))
  WordCount = len(splitSentence)
  for i in range(n):
    firstIndex = random.randint(0,WordCount-1)
    secondIndex = random.randint(0,WordCount-1)
    while (secondIndex == firstIndex and WordCount != 1):
      secondIndex = random.randint(0,WordCount-1)
    splitSentence[firstIndex], splitSentence[secondIndex] = splitSentence[secondIndex], splitSentence[firstIndex]
  return " ".join(splitSentence)

In [63]:
print(eda_RS("ceo moderna sebut efektivitas CEO vaksin covid mungkin akan turun melawan varian omicron", 6))

omicron covid efektivitas sebut ceo mungkin vaksin moderna akan turun melawan varian CEO


In [64]:
def eda_RD(originalSentence, p):
  """
  Paper Methodology -> Randomly remove each word in the sentence with probability p.
  originalSentence -> The sentence on which EDA is to be applied
  p -> Probability of a Word Being Removed
  """
  og = originalSentence
  if (p == 1):
      raise Exception("Always an Empty String Will Be Returned")
  if (p > 1 or p < 0):
    raise Exception("Improper Probability Value")
  splitSentence = list(originalSentence.split(" "))
  lsIndexesRemoved = []
  WordCount = len(splitSentence)
  for i in range(WordCount):
    randomDraw = random.random()
    if randomDraw <= p:
      lsIndexesRemoved.append(i)
  lsRetainingWords = []
  for i in range(len(splitSentence)):
    if i not in lsIndexesRemoved:
      lsRetainingWords.append(splitSentence[i])
  if (lsRetainingWords == []):
    return og
  return " ".join(lsRetainingWords)

In [65]:
print(eda_RD("selanjutnya kita bisa menghitung jumlah kemunculan tiap kata pada text", 0.3))

selanjutnya kita bisa menghitung jumlah kemunculan tiap pada


In [66]:
print(eda_RD("selanjutnya kita bisa menghitung jumlah kemunculan tiap kata pada text", 0.7))

selanjutnya bisa menghitung jumlah tiap kata pada


### Building Dataset

In [67]:
# !wget -q https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv
# !wget -q https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/test.tsv
# !wget -q https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/dev.tsv

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # Load the dataset from a CSV file
# data = pd.read_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/tweet_clean.xlsx')  # Replace 'your_dataset.csv' with your dataset file

# # Split the dataset into train and test sets (80% train, 20% test)
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# # Save the train and test data into separate CSV files
# train_data.to_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/train_data.xlsx', index=False)
# test_data.to_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/test_data.xlsx', index=False)


In [70]:
from tqdm.notebook import tqdm
import numpy as np

In [71]:
def buildTrainTestDatasets(doAug, AugTechs, alpha, NumberOfTrainingSamplesTouse, n_aug):
  df_train = pd.read_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/1. experiment 2/1. Dataset/train_negatif.xlsx',  names = ['label','text'])
  #df_dev = pd.read_csv('dev.tsv', delimiter = '\t', names = ['Sentence', 'Label'])
  df_test = pd.read_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/1. experiment 2/1. Dataset/test_data.xlsx',  names = ['label','text'])
  df_train = df_train[df_train['label'] == 1]

  # print(len(df_train))
  # print(df_train.head())
  # print(len(df_test))
  # print(df_test.head())

  #df_train = pd.concat([df_train, df_dev])
  df_train = df_train.reset_index(drop=True)
  df_train = df_train.loc[:, ["label", "text"]]
  if (NumberOfTrainingSamplesTouse > len(df_train)):
    raise Exception("More Samples Asked For Than Present")

  df_train = df_train.iloc[:NumberOfTrainingSamplesTouse]

  if (doAug):
    if (AugTechs == None):
      AugTechs = ['SR', 'RI', 'RD', 'RS']
    ls_train_labels = df_train['label'].to_list()
    ls_train_sentences = df_train['text'].to_list()
    ls_train_aug_method = list(np.zeros(len(df_train)))

    # print("ls_train_sentences:", ls_train_sentences)

    count = len(ls_train_sentences)
    for i in tqdm(range(count)):
      for iter in range(n_aug):
        ls_train_labels.append(ls_train_labels[i])
        techniqueChosen = random.choice(AugTechs)
        ls_train_aug_method.append(techniqueChosen)
        # print("ls_train_sentences", i, ":", ls_train_sentences[i])
        # print("len", len(ls_train_sentences))
        if (techniqueChosen == 'SR'):
          ls_train_sentences.append(eda_SR(ls_train_sentences[i], int(len(ls_train_sentences[i])*alpha)))
        elif (techniqueChosen == 'RI'):
          ls_train_sentences.append(eda_RI(ls_train_sentences[i], int(len(ls_train_sentences[i])*alpha)))
        elif (techniqueChosen == 'RS'):
          ls_train_sentences.append(eda_RS(ls_train_sentences[i], int(len(ls_train_sentences[i])*alpha)))
        elif (techniqueChosen == 'RD'):
          ls_train_sentences.append(eda_RD(ls_train_sentences[i], alpha))
    df_train = pd.DataFrame(
    {'label': ls_train_labels,
     'text': ls_train_sentences,
    })

  df_train_aug = pd.DataFrame(
      {
          'label': ls_train_labels,
          'method': ls_train_aug_method,
          'text': ls_train_sentences
      }
  )

  target_train = df_train.pop('label')
  target_test = df_test.pop('label')
  feature_names = ['text']
  train_features = df_train[feature_names]
  test_features = df_test[feature_names]
  train_dataset = tf.convert_to_tensor(train_features)
  test_dataset = tf.convert_to_tensor(test_features)
  return train_dataset, target_train, test_dataset, target_test, df_train_aug

In [72]:
  df_train = pd.read_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/1. experiment 2/1. Dataset/train_negatif.xlsx',  names = ['label','text'])
  df_train = df_train[df_train['label'] == -1]
  df_train.to_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/1. experiment 2/1. Dataset/train_eda_negatif.xlsx')
  # print(len(df_train))
  # #df_dev = pd.read_csv('dev.tsv', delimiter = '\t', names = ['Sentence', 'Label'])
  # df_test = pd.read_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/test_data.xlsx',  names = ['label','text'])


## Total Tweet
### =Train Data=
#### netral = 6082
#### positif = 1299
#### negatif = 900

### =Test Data=
#### netral = 1360
#### positif = 490
#### negatif = 347

In [73]:
train_aug_dataset, target_aug_train, test_aug_dataset, target_aug_test, df_train_aug = buildTrainTestDatasets(True, None, 0.04, 500, 7)
# train_aug_dataset, target_aug_train, test_aug_dataset, target_aug_test = buildTrainTestDatasets(True, "RD", 0.05, 500, 16)
print(f"jumlah train aug dataset: {len(train_aug_dataset)}")
print(train_aug_dataset[:5])
# print(len(train_aug_method))
# print(len(train_aug_dataset))

print(df_train_aug.head())

df_train_aug.to_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/1. experiment 2/1. Dataset/train_eda_negatif.xlsx')

# stopp

# train_aug_dataset = tf.squeeze(train_aug_dataset)
# test_aug_dataset = tf.squeeze(test_aug_dataset)
# target_aug_train = tf.squeeze(target_aug_train)
# target_aug_test = tf.squeeze(target_aug_test)

Exception: More Samples Asked For Than Present

In [ ]:
train_nonaug_dataset, target_nonaug_train, test_nonaug_dataset, target_nonaug_test = buildTrainTestDatasets(False, None, None, 500, None)
train_nonaug_dataset = tf.squeeze(train_nonaug_dataset)
test_nonaug_dataset = tf.squeeze(test_nonaug_dataset)
target_nonaug_train = tf.squeeze(target_nonaug_train)
target_nonaug_test = tf.squeeze(target_nonaug_test)

In [ ]:
train_aug = tf.data.Dataset.from_tensor_slices((train_aug_dataset, target_aug_train))
train_non_aug = tf.data.Dataset.from_tensor_slices((train_nonaug_dataset, target_nonaug_train))
test_aug = tf.data.Dataset.from_tensor_slices((test_aug_dataset, target_aug_test))
test_non_aug = tf.data.Dataset.from_tensor_slices((test_nonaug_dataset, target_nonaug_test))

In [ ]:
for example, label in train_aug.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

In [ ]:
for example, label in train_aug.take(10):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

In [ ]:
train_aug, train_non_aug, test_aug, test_non_aug

In [ ]:
for example, label in train_aug.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_aug = train_aug.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_aug = test_aug.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
train_non_aug = train_non_aug.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_non_aug = test_non_aug.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_aug.element_spec

In [ ]:
for example, label in train_aug.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

## RNN Model

Paper Uses -
The architecture used in this paper is as follows: input layer, bi-directional hidden layer with 64 LSTM cells, dropout layer with p=0.5, bi-directional layer of 32 LSTM cells, dropout layer with p=0.5, dense layer of 20 hidden units with ReLU activation, softmax output layer. We initialize this network with random normal weights and train against the categorical crossentropy loss function with the adam optimizer. We use early stopping with a patience of 3 epochs.

Reference for Encoder and some Helper Functions: [Tensorflow Text Classification by RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_classification_rnn)



In [ ]:
VOCAB_SIZE = 5000
encoder_aug = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder_aug.adapt(train_aug.map(lambda text, label: text))

vocab_aug = np.array(encoder_aug.get_vocabulary())

VOCAB_SIZE = 5000
encoder_nonaug = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder_nonaug.adapt(train_non_aug.map(lambda text, label: text))

vocab_nonaug = np.array(encoder_nonaug.get_vocabulary())

In [ ]:
encoded_example = encoder_aug(example)[:3].numpy()
encoded_example

for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab_aug[encoded_example[n]]))
  print()

In [ ]:
model_aug = tf.keras.Sequential([
    encoder_aug,
    tf.keras.layers.Embedding(len(encoder_aug.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, kernel_initializer='normal', activation='sigmoid')
])
model_aug.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model_nonaug = tf.keras.Sequential([
    encoder_nonaug,
    tf.keras.layers.Embedding(len(encoder_nonaug.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, kernel_initializer='normal', activation='sigmoid')
])
model_nonaug.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

history_aug = model_aug.fit(
      train_aug,
      epochs=50,
      callbacks=callbacks,
      validation_data=test_aug,
      validation_steps=5
)

history_non_aug = model_nonaug.fit(
      train_non_aug,
      epochs=50,
      callbacks=callbacks,
      validation_data=test_non_aug,
      validation_steps=5
)

In [ ]:
test_aug_loss, test_aug_acc = model_aug.evaluate(test_aug)

print('Test Augmented Loss:', test_aug_loss)
print('Test Augmented Accuracy:', test_aug_acc)

test_non_aug_loss, test_non_aug_acc = model_nonaug.evaluate(test_non_aug)

print('Test Non Augmented Loss:', test_non_aug_loss)
print('Test Non Augmented Accuracy:', test_non_aug_acc)

In [ ]:
## Augmented Data Training
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history_aug, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history_aug, 'loss')
plt.ylim(0, None)

In [ ]:
## Non Augmented Data Training
plt.figure(figsize=(16, 8))
plt.title("Non Augmented Data Training")
plt.subplot(1, 2, 1)
plot_graphs(history_non_aug, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history_non_aug, 'loss')
plt.ylim(0, None)